## punto 1.1

In [ ]:
import bs4 # pip install 
from bs4 import BeautifulSoup

In [ ]:
import requests
url = 'https://www.findamasters.com/masters-degrees/msc-degrees/'
result = requests.get(url)
result.text

In [ ]:
def extract_masters(this_url):
    result_url=requests.get(this_url)
    result_soup=BeautifulSoup(result_url.text)
    result_links=result_soup.find_all('a',{'class':'courseLink'})
    result_list=[]
    for item in result_links:
        result_list.append((item['href'],item.text))
    return result_list

In [ ]:
# to extract all the masters we have to do a for loop for the first 400 pages:
"""
num_pages = 400
pref = 'https://www.findamasters.com'
test_lst_all = []

for i in range(1, num_pages + 1):
    test_lst = extract_masters(pref + '/masters-degrees/msc-degrees/?PG=' + str(i))
    test_lst_all.extend(test_lst)
# creating the txt file of the fisrt 400 pages of ms
with open('masters_urls.txt', 'w') as f:
    for item in test_lst_all:
        f.write(item[0] + '\n')
"""
# file created 